<a href="https://colab.research.google.com/github/AswathyS9778/project/blob/main/pro12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode

     |████████████████████████████████| 235 kB 5.3 MB/s 


In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import unidecode
import plotly.express as px

In [ ]:
first_product = pd.read_csv('nutrients_csvfile (1).csv')
first_product['product'] = '1 - product'

second_product = pd.read_csv('nutrition2.csv')
second_product['product'] = '2 - product'

orders = pd.concat([first_product,second_product])
orders.drop('Measure',axis=1,inplace=True)

In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 95
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           171 non-null    float64
 1   Food         171 non-null    object 
 2   Quantity     171 non-null    float64
 3   Grams        171 non-null    object 
 4   Calories     170 non-null    object 
 5   Protein      171 non-null    object 
 6   Fat          171 non-null    object 
 7   Sat.Fat      170 non-null    object 
 8   Fiber        171 non-null    object 
 9   Carbs        171 non-null    object 
 10  Category     171 non-null    object 
 11  Unnamed: 12  0 non-null      float64
 12  Unnamed: 13  0 non-null      float64
 13  Unnamed: 14  0 non-null      float64
 14  Unnamed: 15  0 non-null      float64
 15  product      299 non-null    object 
dtypes: float64(6), object(10)
memory usage: 39.7+ KB


In [ ]:
orders.sample(5)

,id,Food,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,product
41,42.0,Whole-wheat,454,"1,100",48,14,10,67.5,216,"Breads, cereals, fastfood,grains",NaN,NaN,NaN,NaN,NaN,2 - product
22,23.0,Olive oil,14,125,0,14,3,0,0,"Fats, Oils, Shortenings",NaN,NaN,NaN,NaN,NaN,1 - product
113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 - product
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 - product
52,53.0,Bean soups,250,190,8,5,4,0.6,30,Soups,NaN,NaN,NaN,NaN,NaN,2 - product


In [ ]:
def format_columns(column):
    new_column = ' '.join(column.split())
    new_column = new_column.replace(' ','_')
    return new_column.lower()

orders.columns = [format_columns(c) for c in orders.columns]

In [ ]:
orders.isna().mean()

id             0.428094
food           0.428094
quantity       0.428094
grams          0.428094
calories       0.431438
protein        0.428094
fat            0.428094
sat.fat        0.431438
fiber          0.428094
carbs          0.428094
category       0.428094
unnamed:_12    1.000000
unnamed:_13    1.000000
unnamed:_14    1.000000
unnamed:_15    1.000000
product        0.000000
dtype: float64

In [ ]:
def stats_summary(data):
    
   # sales = data['Carbs'].sum()
    quantity = data['quantity'].sum()
    orders_count = len(data['id'].unique())
    
    avg_price_order = quantity/ orders_count
    #avg_price_food = sales / quantity
    count_food_orders = quantity / orders_count
    
    return {'avg_price_order': avg_price_order,
      
            'count_food_orders': count_food_orders
           }

def print_summary(data):
  
    stats = stats_summary(data)
    
    for key, value in stats.items():
        print(f'{key}: {value:.3}')

In [ ]:
print('\n1 - product\n')
print_summary(orders[orders['product']=='1 - product'])
print('\n2 - product\n')
print_summary(orders[orders['product']=='2 - product'])

print('\nTotal\n')
print_summary(orders)


1 - product

avg_price_order: 1.24
count_food_orders: 1.24

2 - product

avg_price_order: 1.21
count_food_orders: 1.21

Total

avg_price_order: 2.16
count_food_orders: 2.16


In [ ]:
items_frequence = orders.groupby('food').size().reset_index(name='quantity')
print(items_frequence.describe())
print(f'\ntotal unique orders: {len(orders["id"].unique())}')

         quantity
count  168.000000
mean     1.017857
std      0.172103
min      1.000000
25%      1.000000
50%      1.000000
75%      1.000000
max      3.000000

total unique orders: 97


In [ ]:
#convert the dataframe to list items in order
item_list = orders.groupby('id')['food'].unique()

# transform the values of the data set to 1 if that item belongs to that order, otherwise 0
te = TransactionEncoder()
oht_orders = te.fit(item_list).transform(item_list, sparse=True)

In [ ]:
sparse_df_items = pd.DataFrame.sparse.from_spmatrix(oht_orders, columns=te.columns_)

In [ ]:
frequent_itemsets = apriori(sparse_df_items, min_support=0.02209,max_len=11, use_colnames=True)

In [ ]:
frequent_itemsets_plot = frequent_itemsets.copy()
frequent_itemsets_plot['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets_plot['support'] = (frequent_itemsets_plot['support'] * 100).round(2)
frequent_itemsets_plot["itemsets"] = frequent_itemsets_plot["itemsets"].apply(lambda x: ', '.join(list(x))).astype("str")

In [ ]:
frequent_itemsets_plot.groupby('length')['support'].describe()

,count,mean,std,min,25%,50%,75%,max
length,,,,,,,,
1,1.0,3.12,NaN,3.12,3.12,3.12,3.12,3.12


In [ ]:
top_20_frequence = frequent_itemsets_plot.sort_values('support',ascending=False).head(16).sort_values('support')
fig = px.bar(top_20_frequence, x="support", y="itemsets", orientation='h', text='support')
fig.update_traces(textposition="outside")
fig.show()

In [ ]:
market_basket_rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
market_basket_rules.groupby('antecedents').size().sort_values(ascending=False)

Series([], dtype: int64)

In [ ]:
best_item_recommendations = market_basket_rules.sort_values(['confidence','lift'],ascending='False').drop_duplicates(subset=['antecedents'])
top_20_frequence_items = frequent_itemsets.sort_values('support',ascending=False).head(20)['itemsets']
best_item_recommendations[best_item_recommendations['antecedents'].isin(top_20_frequence_items)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
